### LightGBM
* original by https://github.com/KGDELAB

In [10]:
#데이터 전처리

import os
import pandas as pd
import numpy as np

base_dir = 'Dataset'
water_dir = base_dir + '/water_data'
rf_dir = base_dir + '/rf_data'
water_list = os.listdir(water_dir)
rf_list = os.listdir(rf_dir)

submission = pd.read_csv(base_dir + '/sample_submission.csv')
submission['ymdhm'] = pd.to_datetime(submission['ymdhm'])
submission.set_index(keys=['ymdhm'], inplace=True)

water = []
for i in range(len(water_list)):
    file_dir = water_dir + '/' + water_list[i]
    df = pd.read_csv(file_dir)
    water.append(df)

water = pd.concat(water, axis = 0)
    
rf = []
for i in range(len(rf_list)):
    file_dir = rf_dir + '/' + rf_list[i]
    df = pd.read_csv(file_dir)
    rf.append(df)
    
rf = pd.concat(rf, axis = 0)

df = pd.merge(water, rf)
df['ymdhm'] = pd.to_datetime(df['ymdhm'])
df.set_index(keys=['ymdhm'], inplace=True)
cutoff = '2022-06-01'
df.loc[cutoff:, submission.columns] = submission[submission.columns]
submission.reset_index(inplace=True)
submission['ymdhm']=submission['ymdhm'].dt.strftime('%Y-%m-%d %H:%M')
df.reset_index(inplace=True)
df['ymdhm']=df['ymdhm'].dt.strftime('%Y-%m-%d %H:%M')

df = df.replace(' ', np.NaN)
df = df.replace('##########', np.NaN)
df['ymdhm'] = pd.to_datetime(df['ymdhm'])
df.set_index('ymdhm',inplace=True)
df = df.astype('float')
df.reset_index(inplace=True)

In [11]:
df

,ymdhm,swl,inf,sfw,ecpc,tototf,tide_level,wl_1018662,fw_1018662,wl_1018680,fw_1018680,wl_1018683,fw_1018683,wl_1019630,fw_1019630,rf_10184100,rf_10184110,rf_10184140
0,2012-01-01 00:00:00,24.950,232.00,224.23,19.77,134.00,NaN,278.7,269.40,273.2,0.0,261.0,175.16,253.3,310.31,0.0,0.0,0.0
1,2012-01-01 00:10:00,24.929,134.40,223.49,20.51,134.40,NaN,279.7,274.78,273.2,0.0,261.0,176.60,258.3,356.89,0.0,0.0,0.0
2,2012-01-01 00:20:00,24.929,128.30,223.49,20.51,128.30,NaN,279.7,274.78,273.2,0.0,261.0,179.49,261.3,386.43,0.0,0.0,0.0
3,2012-01-01 00:30:00,24.929,134.40,223.49,20.51,134.40,NaN,278.7,269.40,273.2,0.0,261.0,173.72,262.3,396.55,0.0,0.0,0.0
4,2012-01-01 00:40:00,24.937,232.70,223.74,20.26,134.40,NaN,279.7,274.78,273.2,0.0,261.0,76.99,262.3,396.55,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537403,2022-07-18 23:10:00,25.040,259.23,212.86,31.14,259.23,510.0,0.0,319.84,0.0,NaN,0.0,-456.41,0.0,974.40,0.0,0.0,0.0
537404,2022-07-18 23:20:00,25.040,260.46,212.86,31.14,260.46,492.0,0.0,314.01,0.0,NaN,0.0,-717.30,0.0,1006.88,0.0,0.0,0.0
537405,2022-07-18 23:30:00,25.040,259.37,212.86,31.14,259.37,475.0,0.0,387.55,0.0,NaN,0.0,-843.37,0.0,1039.90,0.0,0.0,0.0
537406,2022-07-18 23:40:00,25.040,259.13,212.86,31.14,259.13,458.0,0.0,454.91,0.0,NaN,0.0,-1023.37,0.0,1073.46,0.0,0.0,0.0


In [3]:
#df['f1'] = df['tototf'] - df['inf']

In [4]:
#df['sum_fw'] = df['fw_1018662'] + df['fw_1018680'] + df['fw_1018683'] + df['fw_1019630']

In [12]:
df.fillna(df.mean(), inplace=True)

<ipython-input-12-bf934754a5b1>:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  df.fillna(df.mean(), inplace=True)


In [13]:
df.columns

Index(['ymdhm', 'swl', 'inf', 'sfw', 'ecpc', 'tototf', 'tide_level',
       'wl_1018662', 'fw_1018662', 'wl_1018680', 'fw_1018680', 'wl_1018683',
       'fw_1018683', 'wl_1019630', 'fw_1019630', 'rf_10184100', 'rf_10184110',
       'rf_10184140'],
      dtype='object')

In [15]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler

columns = ['swl', 'inf', 'sfw', 'ecpc', 'tototf', 'tide_level',
       'fw_1018662', 'fw_1018680', 'fw_1018683','fw_1019630',
        'rf_10184100', 'rf_10184110', 'rf_10184140']
std = MinMaxScaler()
df[columns] = std.fit_transform(df[columns])

In [16]:
df

,ymdhm,swl,inf,sfw,ecpc,tototf,tide_level,wl_1018662,fw_1018662,wl_1018680,fw_1018680,wl_1018683,fw_1018683,wl_1019630,fw_1019630,rf_10184100,rf_10184110,rf_10184140
0,2012-01-01 00:00:00,0.496913,0.254178,0.924206,0.075794,0.041969,0.399610,278.7,0.028642,273.2,0.0,261.0,0.128345,253.3,0.010394,0.0,0.0,0.0
1,2012-01-01 00:10:00,0.496495,0.251352,0.921156,0.078844,0.041984,0.399610,279.7,0.029214,273.2,0.0,261.0,0.128395,258.3,0.011954,0.0,0.0,0.0
2,2012-01-01 00:20:00,0.496495,0.251176,0.921156,0.078844,0.041756,0.399610,279.7,0.029214,273.2,0.0,261.0,0.128496,261.3,0.012944,0.0,0.0,0.0
3,2012-01-01 00:30:00,0.496495,0.251352,0.921156,0.078844,0.041984,0.399610,278.7,0.028642,273.2,0.0,261.0,0.128295,262.3,0.013283,0.0,0.0,0.0
4,2012-01-01 00:40:00,0.496654,0.254198,0.922186,0.077814,0.041984,0.399610,279.7,0.029214,273.2,0.0,261.0,0.124938,262.3,0.013283,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537403,2022-07-18 23:10:00,0.498705,0.254966,0.877344,0.122656,0.046645,0.607101,0.0,0.034005,0.0,0.0,0.0,0.106425,0.0,0.032638,0.0,0.0,0.0
537404,2022-07-18 23:20:00,0.498705,0.255002,0.877344,0.122656,0.046691,0.585799,0.0,0.033385,0.0,0.0,0.0,0.097370,0.0,0.033726,0.0,0.0,0.0
537405,2022-07-18 23:30:00,0.498705,0.254970,0.877344,0.122656,0.046650,0.565680,0.0,0.041204,0.0,0.0,0.0,0.092994,0.0,0.034832,0.0,0.0,0.0
537406,2022-07-18 23:40:00,0.498705,0.254963,0.877344,0.122656,0.046642,0.545562,0.0,0.048366,0.0,0.0,0.0,0.086747,0.0,0.035957,0.0,0.0,0.0


In [17]:
# data, target 분리하기

target = df.loc[:,submission.columns]
target.rename(columns={'wl_1018662':'target1','wl_1018680':'target2',
                       'wl_1018683':'target3','wl_1019630':'target4'},inplace=True)
data = df.drop(['ymdhm'],axis=1)
data.index += 1

df = pd.concat((data, target), axis = 1)
df = df.iloc[1:-1]
df.reset_index(inplace=True, drop=True)

In [18]:
import lightgbm as lgb
import tensorflow as tf

tmp = ['ymdhm','target1','target2','target3','target4']
data = df.drop(tmp,axis=1)
train_data=data.iloc[:-len(submission),:].iloc[:-160000,:]
valid_data=train_data.iloc[-160000:,:]
test_data=data.iloc[-len(submission):,:]

with tf.device("/device:GPU:0"):
    for i in range(len(tmp)-1):
        target = df.loc[:,[tmp[i+1]]]

        train_target=target.iloc[:-len(submission),:].iloc[:-160000,:]
        valid_target=train_target.iloc[-160000:,:]

        train_ds = lgb.Dataset(train_data, label = train_target)
        valid_ds = lgb.Dataset(valid_data, label = valid_target)

        params = {'learning_rate': 0.01, 
                  'max_depth': -1,
                  'boosting': 'gbdt', 
                  'metric': 'rmse',
                  'objective': 'regression',
                  'is_training_metric' : True,
                  'num_leaves': 200,
                  'feature_fraction': 0.9,
                  'bagging_fraction': 0.7,
                  'bagging_freq': 5,
                  'seed': 2022}

        model = lgb.train(params, train_ds, 5000, valid_ds, verbose_eval = 100, early_stopping_rounds = 100)
        y_pred=model.predict(test_data)

        submission.iloc[:,i+1:i+2] = y_pred

C:\Users\Gwangwon\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Gwangwon\anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.013130 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3359
[LightGBM] [Info] Number of data points in the train set: 370495, number of used features: 16
[LightGBM] [Info] Start training from score 314.802003
Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 23.4071
[200]	valid_0's rmse: 9.67357
[300]	valid_0's rmse: 5.43712
[400]	valid_0's rmse: 4.37592
[500]	valid_0's rmse: 4.10059
[600]	valid_0's rmse: 3.95814
[700]	valid_0's rmse: 3.86085
[800]	valid_0's rmse: 3.75623
[900]	valid_0's rmse: 3.67061
[1000]	valid_0's rmse: 3.58752
[1100]	valid_0's rmse: 3.51846
[1200]	valid_0's rmse: 3.44797
[1300]	valid_0's rmse: 3.39038
[1400]	valid_0's rmse: 3.32581
[1500]	valid_0's rmse: 3.26408
[1600]	valid_0's rmse: 3.20852
[1700]	valid_0's rmse: 3.16653
[1800]	valid_0's rmse: 3.12082
[1900]	valid_0's rmse: 3.06998
[2000]	val

In [19]:
submission

,ymdhm,wl_1018662,wl_1018680,wl_1018683,wl_1019630
0,2022-06-01 00:00,281.095628,276.051856,274.586434,268.746165
1,2022-06-01 00:10,264.562541,252.238404,249.034868,185.168915
2,2022-06-01 00:20,264.790372,252.484878,249.120300,191.898458
3,2022-06-01 00:30,262.067126,252.141042,247.805569,181.665243
4,2022-06-01 00:40,260.838715,252.664870,247.732077,190.895004
...,...,...,...,...,...
6907,2022-07-18 23:10,254.663641,252.428177,249.456722,226.439340
6908,2022-07-18 23:20,256.337287,253.446883,249.895138,217.248262
6909,2022-07-18 23:30,256.784038,252.626684,249.844229,223.943765
6910,2022-07-18 23:40,261.138640,252.737918,249.182011,235.215980


In [ ]:
# # R2 SCORE
# from sklearn.metrics import r2_score

# r2score = r2_score(submission[['wl_1018662','wl_1018680','wl_1018683','wl_1019630']])
# print(r2score)

In [20]:
submission.to_csv('gw_test5.csv', index=False)